# Spark demo

Illustrates how to use the ammonite-spark library, that adds Spark support to (my [fork](https://github.com/alexarchambault/ammonite-shell) of) the ammonite REPL or a notebook.

The examples come mostly from the spark-repl test suite: https://github.com/apache/spark/blob/master/repl/scala-2.11/src/test/scala/org/apache/spark/repl/ReplSuite.scala.

We start from a bare scala 2.10 jupyter scala notebook. First, let's fetch ammonite-spark.

Notice the `1.3` in the dependency name, which corresponds to our spark version, and the `2.10.5` which is the full current scala version.

In [1]:
load.ivy("com.github.alexarchambault" % "ammonite-spark_1.3_2.11.6" % "0.3.1-SNAPSHOT")

:::: WARNINGS


:: problems summary ::

	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT from sonatype-snapshots, using Mon Jun 01 15:32:30 CEST 2015

	Sorting results from com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT, using Mon Jun 01 15:32:30 CEST 2015 and null

	Sorting results from com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT, using Mon Jun 01 10:20:55 CEST 2015 and Mon Jun 01 10:20:55 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT from sonatype-snapshots, using Mon Jun 01 14:46:12 CEST 2015

	Sorting results from com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT, using Mon Jun 01 14:46:12 CEST 2015 and null

	Sorting results from com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT, using Mon Jun 01 10:18:43 CEST 2015 and Mon Jun 01 10:18:43 CES

pe-snapshots for com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT

	Unable to reparse com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT from sonatype-snapshots, using Mon Jun 01 02:54:01 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-spark_1.3_2.11.6;0.3.1-SNAPSHOT from sonatype-snapshots, using Mon Jun 01 14:46:23 CEST 2015

	Sorting results from com.github.alexarchambault#ammonite-spark_1.3_2.11.6;0.3.1-SNAPSHOT, using Mon Jun 01 14:46:23 CEST 2015 and null

	Sorting results from com.github.alexarchambault#ammonite-spark_1.3_2.11.6;0.3.1-SNAPSHOT, using Mon Jun 01 10:19:06 CEST 2015 and Mon Jun 01 10:19:06 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault#ammonite-spark_1.3_2.11.6;0.3.1-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-shell-api_2.11.6;0.3.1-SNAPSHOT from sonatype-snapshots, using Mon

:::: ERRORS
	unknown resolver null

	unknown resolver null

	unknown resolver null

	unknown resolver null

	unknown resolver null

	unknown resolver null

	unknown resolver null

	unknown resolver null

	unknown resolver null




	Sorting results from com.github.alexarchambault#ammonite-shell-api_2.11.6;0.3.1-SNAPSHOT, using Mon Jun 01 10:18:43 CEST 2015 and Mon Jun 01 10:18:43 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault#ammonite-shell-api_2.11.6;0.3.1-SNAPSHOT



Now, we'll create a handle, of type `ammonite.spark.Spark`. Upon creation, it gets an implicit `ammonite.api.Interpreter`, that one can find when running code from a notebook, and uses it later to initialize spark.

Note the `@transient` annotation added to it, so that it will not get serialized when running closures on a cluster.

In [2]:
@transient val Spark = new ammonite.spark.Spark

log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Spark: ammonite.spark.Spark = Spark(uninitialized)

A `SparkContext` is accessible through the `sc` method of the spark handle. It is *lazily* initialized, which means it is not yet, as we didn't call the `sc` method.

Before that, we'll setup the spark config, through a `SparkConf`.

In [3]:
Spark.withConf(_
  .setMaster("local")
)

In [4]:
import Spark.sc

import Spark.sc

In [5]:
Spark.start() // equivalent to just calling sc, triggers its initialization

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
15/06/01 19:38:03 INFO Spark$SparkContext: Running Spark version 1.3.1
15/06/01 19:38:04 WARN Utils: Your hostname, pc-ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.0.15 instead (on interface eth0)
15/06/01 19:38:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
15/06/01 19:38:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/06/01 19:38:04 INFO SecurityManager: Changing view acls to: alexandre
15/06/01 19:38:04 INFO SecurityManager: Changing modify acls to: alexandre
15/06/01 19:38:04 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users with view permissions: Set(alexandre); users with modify permissions: Set(alexandre)
15/06/01 19:38:04 INFO Slf4jLogger: Slf4jLogger started
15/06/01 19:38:04 INFO Remoting: Starting remoting
15/06/01 19:38:04 INFO Remo

Now the examples:

### Accumulator

In [6]:
val accum = sc.accumulator(0)

accum: org.apache.spark.Accumulator[Int] = 0

In [7]:
sc.parallelize(1 to 10).foreach(x => accum += x)

In [8]:
accum.value

res7: Int = 55

### Parallel function

In [9]:
def double(x: Int) = x + x

defined function double

In [10]:
sc.parallelize(1 to 10).map(x => double(x)).collect().reduceLeft(_+_)

res9: Int = 110

### Mutable variable

In [11]:
var v = 7

v: Int = 7

In [12]:
def getV() = v

defined function getV

In [13]:
sc.parallelize(1 to 10).map(x => getV()).collect().reduceLeft(_+_)

res12: Int = 70

In [14]:
v = 10

In [15]:
sc.parallelize(1 to 10).map(x => getV()).collect().reduceLeft(_+_)

res14: Int = 100

### Defining your own classes is ok

and more complex definions should too

In [16]:
case class Sum(exp: String, exp2: String)

defined class Sum

In [17]:
val a = Sum("A", "B")

a: Sum = Sum("A", "B")

In [18]:
def b(a: Sum): String = a match { case Sum(_, _) => "Found Sum" }

defined function b

In [19]:
b(a)

res18: String = "Found Sum"

### SparkSQL :|

Spark SQL doesn't work from here because of https://issues.apache.org/jira/browse/SPARK-5281. It should be fixed in the next Spark releases.

In [20]:
import Spark.sqlContext
import sqlContext.implicits._

import Spark.sqlContext
import sqlContext.implicits._

In [21]:
case class TestCaseClass(value: Int)

defined class TestCaseClass

In [22]:
sc.parallelize(1 to 10).map(x => TestCaseClass(x)).toDF().collect()

: 

### Unused non serializable things are fine

If they are not used in parallel calculations, they do not prevent serialization.

In [23]:
class TestClass() { def testMethod = 3; override def toString = "TestClass" }

defined class TestClass

In [24]:
// not serializable
val t = new TestClass

t: TestClass = TestClass

In [25]:
import t.testMethod

import t.testMethod

In [26]:
// serializable
case class TestCaseClass(value: Int)

defined class TestCaseClass

In [27]:
// some parallel calculations with the serializable class
sc.parallelize(1 to 10).map(x => TestCaseClass(x)).collect()

res26: Array[TestCaseClass] = Array(
  TestCaseClass(1),
  TestCaseClass(2),
  TestCaseClass(3),
  TestCaseClass(4),
  TestCaseClass(5),
  TestCaseClass(6),
  TestCaseClass(7),
  TestCaseClass(8),
  TestCaseClass(9),
  TestCaseClass(10)
)